In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import sklearn
import argparse
import sklearn.metrics

from dataset import PDDDataset
from models import PhenoProfiler
from utils import *

# define hyperparameters
model = PhenoProfiler().cuda()
model_path = "result/PhenoProfiler/last.pt"
save_path = "Fig3/BBBC022/PhenoProfiler_test/"

In [2]:
# Load metadata of index data
meta = pd.read_csv(os.path.join(f"../dataset/bbbc022/profiling.csv"))

MATRIX_FILE = save_path+"/cos_efn128combinedcellsout_conv6a_1e-2_e30.csv"
REG_PARAM = 1e-2

meta["broad_sample"] = meta["Treatment"].str.split("@", expand=True)[0]

features = np.load(save_path+"DeepProfiler_Alltrain_22test.npy").T
print(features.shape)
meta

(66558, 672)


,Metadata_Plate,Metadata_Well,Metadata_Site,Plate_Map_Name,DNA,ER,RNA,AGP,Mito,broad_sample_Replicate,Treatment,Compound,Concentration,Replicate_Use,broad_sample
0,20585,A01,1,H-BIOA-002-1,20585/IXMtest_A01_s1_w164FBEEF7-F77C-4892-86F5...,20585/IXMtest_A01_s1_w3FD54227A-6BE4-460B-8E2C...,20585/IXMtest_A01_s1_w22CA7BDF3-1425-4AE6-8E57...,20585/IXMtest_A01_s1_w47457436C-5BE7-4DA2-B966...,20585/IXMtest_A01_s1_w58DA6AF62-1799-4B85-BE17...,1,BRD-K98763141@8.858401198,BRD-K98763141,8.858401,Training,BRD-K98763141
1,20585,A01,2,H-BIOA-002-1,20585/IXMtest_A01_s2_w1AA6B1894-F561-42EE-9D1D...,20585/IXMtest_A01_s2_w3A597237B-C3D7-43AE-8399...,20585/IXMtest_A01_s2_w246FFAEE1-BEB6-4C81-913B...,20585/IXMtest_A01_s2_w46657239A-5AFE-4C29-BB9B...,20585/IXMtest_A01_s2_w50F1562CD-EBCF-408E-9D8D...,1,BRD-K98763141@8.858401198,BRD-K98763141,8.858401,Training,BRD-K98763141
2,20585,A01,3,H-BIOA-002-1,20585/IXMtest_A01_s3_w108F3A6FF-A865-40A1-8488...,20585/IXMtest_A01_s3_w30B946780-C923-4A40-8EC5...,20585/IXMtest_A01_s3_w22AD5BD08-58A3-466D-A364...,20585/IXMtest_A01_s3_w47B5F2FF6-A5A2-4101-AC2A...,20585/IXMtest_A01_s3_w5489716D4-2D06-45BE-8D68...,1,BRD-K98763141@8.858401198,BRD-K98763141,8.858401,Training,BRD-K98763141
3,20585,A01,4,H-BIOA-002-1,20585/IXMtest_A01_s4_w189967984-D8CB-4449-9CE5...,20585/IXMtest_A01_s4_w3F582BADD-2249-48D6-BC19...,20585/IXMtest_A01_s4_w231D1EC99-22A8-47A8-9691...,20585/IXMtest_A01_s4_w47E34AB5F-23B4-42E1-8B5B...,20585/IXMtest_A01_s4_w5D0566314-B9B2-4299-98AC...,1,BRD-K98763141@8.858401198,BRD-K98763141,8.858401,Training,BRD-K98763141
4,20585,A01,5,H-BIOA-002-1,20585/IXMtest_A01_s5_w1DCC72C3D-6ABF-4D49-897E...,20585/IXMtest_A01_s5_w3EB7ABD45-EE05-41A0-979E...,20585/IXMtest_A01_s5_w2B3F07F02-876D-455B-94E4...,20585/IXMtest_A01_s5_w4AFA01300-7373-42C4-872F...,20585/IXMtest_A01_s5_w513201EB0-3500-4A0C-8FAC...,1,BRD-K98763141@8.858401198,BRD-K98763141,8.858401,Training,BRD-K98763141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66553,20646,P24,5,H-BIOA-006-1,20646/IXMtest_P24_s5_w165703D98-F910-4E5C-93C6...,20646/IXMtest_P24_s5_w36BAFF86A-1994-4895-818F...,20646/IXMtest_P24_s5_w23EE5080A-0B6E-4AE1-A413...,20646/IXMtest_P24_s5_w4FC9030FD-81D1-43DF-AA8B...,20646/IXMtest_P24_s5_w5811D5170-11AC-4A60-8AFF...,4,BRD-K08832567@5,BRD-K08832567,5.000000,Training,BRD-K08832567
66554,20646,P24,6,H-BIOA-006-1,20646/IXMtest_P24_s6_w184D3491C-2361-479E-A11B...,20646/IXMtest_P24_s6_w3CC6BB025-8969-4F85-B30E...,20646/IXMtest_P24_s6_w2F9235A94-86DA-4703-B46B...,20646/IXMtest_P24_s6_w4BD3DA210-B3C7-4371-B06D...,20646/IXMtest_P24_s6_w5300E0A76-48E7-489A-AEC7...,4,BRD-K08832567@5,BRD-K08832567,5.000000,Training,BRD-K08832567
66555,20646,P24,7,H-BIOA-006-1,20646/IXMtest_P24_s7_w1B04E085D-04AB-42F7-8D08...,20646/IXMtest_P24_s7_w329876479-D0F9-4A3F-BD44...,20646/IXMtest_P24_s7_w209CDD7FD-92F0-40B4-85C6...,20646/IXMtest_P24_s7_w4DBB3B5B7-0EF6-4418-97D7...,20646/IXMtest_P24_s7_w55DE4D5D3-E291-4FEB-A3BE...,4,BRD-K08832567@5,BRD-K08832567,5.000000,Training,BRD-K08832567
66556,20646,P24,8,H-BIOA-006-1,20646/IXMtest_P24_s8_w166C6499D-6DE8-4BDB-90CE...,20646/IXMtest_P24_s8_w3F65034AA-2A45-470A-8F98...,20646/IXMtest_P24_s8_w23695868E-F144-4CDF-B1A6...,20646/IXMtest_P24_s8_w4D4765757-C520-4B74-9301...,20646/IXMtest_P24_s8_w5090B2268-B603-491F-B313...,4,BRD-K08832567@5,BRD-K08832567,5.000000,Training,BRD-K08832567


In [88]:
# 用序号的方式来指定 control Wells，

# 提取 Compound 列中值为 'DMSO' 的所有行号
dmso_indices = meta.index[meta['Compound'] == 'DMSO'].tolist()

# 打印结果
print(len(dmso_indices), dmso_indices[:10])

11519 [99, 100, 101, 102, 103, 104, 105, 106, 107, 108]


In [89]:
# 提取 meta 中的第一列 'Metadata_Plate'
metadata_plate = meta['Metadata_Plate'].values.reshape(-1, 1)

# 将 'Metadata_Plate' 列融合到 features 的第一列
combined_features = np.hstack((metadata_plate, features))

# 将 combined_features 转换为 DataFrame
combined_df = pd.DataFrame(combined_features, columns=['Metadata_Plate'] + [f'{i}' for i in range(1, features.shape[1]+1)])

# 打印结果的形状以确认
print(combined_df.shape)

# 只保留dmso_indices对应的行
filtered_combined_df = combined_df.iloc[dmso_indices]

# 打印结果的形状以确认
print(filtered_combined_df.shape)

(66558, 673)
(11519, 673)


In [90]:
# control well 对应的 特征
control_df = filtered_combined_df.groupby(["Metadata_Plate"]).mean().reset_index()
control_df

,Metadata_Plate,1,2,3,4,5,6,7,8,9,...,663,664,665,666,667,668,669,670,671,672
0,20585.0,0.235657,0.117635,0.369480,-0.223070,0.246334,0.165181,0.305182,-0.329459,-0.377642,...,0.220398,-0.060135,-0.024452,0.225718,-0.073866,0.050306,-0.135237,0.660830,-0.026316,0.101224
1,20586.0,0.346511,0.433381,-0.171312,-0.019696,0.434148,0.509653,0.242991,0.038223,-0.317462,...,0.016890,0.183222,0.217492,-0.151231,-0.049471,0.056427,-0.307830,0.347055,0.074477,-0.029227
2,20589.0,0.348527,0.157097,0.235231,0.276551,0.358908,0.525940,0.243307,-0.073822,-0.053151,...,-0.187421,0.442398,0.190380,-0.464546,-0.073251,0.048308,-0.183703,0.517527,0.086791,0.023982
3,20590.0,0.241084,0.102837,-0.133951,0.089500,0.492852,0.251499,0.355607,0.163072,0.064227,...,-0.005099,0.240188,-0.041401,-0.249226,-0.062023,-0.022323,-0.197770,0.316286,0.012174,0.176136
4,20591.0,0.144066,0.203775,0.039854,0.075001,0.397288,0.156142,0.164007,-0.074479,-0.043698,...,-0.108546,0.023933,0.098350,-0.025839,-0.063702,-0.085195,-0.202454,0.198735,-0.042299,0.273255
5,20592.0,0.205523,0.198740,0.094164,-0.048429,0.425437,0.109686,0.359839,0.005699,-0.024430,...,0.120501,-0.016417,0.096845,-0.153142,-0.061905,0.024697,-0.291918,0.320780,-0.250126,0.260745
6,20593.0,0.157658,0.284577,0.163802,-0.126437,0.607684,0.024755,0.389352,-0.049811,-0.144042,...,0.085946,0.006386,0.157926,-0.196755,-0.062864,0.018836,-0.267953,0.402371,-0.163542,0.167819
7,20594.0,0.508253,0.320388,0.470747,0.107963,0.629620,0.331666,0.337122,-0.114508,0.185723,...,0.018643,0.038782,0.341104,-0.232825,-0.072397,0.078572,-0.309592,0.299001,0.204282,-0.058038
8,20595.0,0.140626,0.367599,-0.259904,0.190319,0.787387,-0.036552,0.403886,0.094924,0.061201,...,-0.181520,0.095859,-0.051217,-0.489441,-0.062818,-0.020455,-0.099518,0.165979,-0.268327,0.200467
9,20596.0,0.276807,0.317691,0.196789,0.218744,0.272406,0.476010,0.145139,-0.277804,0.050929,...,-0.111677,-0.009842,0.312098,-0.020394,-0.055644,0.002661,-0.216221,0.247225,0.060886,0.010526


In [ ]:
# 减去 control_df 中相同 Metadata_Plate 的行对应的后面 672 特征维度
def subtract_control_features(row, control_df):
    plate = row['Metadata_Plate']
    control_row = control_df[control_df['Metadata_Plate'] == plate]
    if not control_row.empty:
        row.iloc[1:] = row.iloc[1:] - control_row.iloc[0, 1:]
    return row

adjusted_combined_df = combined_df.apply(subtract_control_features, axis=1, control_df=control_df)
# adjusted_combined_df

In [92]:
# 移除第一列 'Metadata_Plate'，然后变成 (66558, 672)
adjusted_combined_df = adjusted_combined_df.drop(columns=['Metadata_Plate'])

# 保存为 .npy 文件
save_path = save_path+"DeepProfiler_Alltrain_22test_0.6.npy"
np.save(save_path, adjusted_combined_df.values)

# 打印结果的形状以确认
print(adjusted_combined_df.shape)

(66558, 672)
